# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,isafjordur,66.0755,-23.1240,1.29,87,100,2.34,IS,1699739709
1,1,coronel suarez,-37.4547,-61.9334,21.42,45,1,4.06,AR,1699739709
2,2,badger,64.8000,-147.5333,-9.07,88,75,1.54,US,1699739710
3,3,mejit,10.2753,170.8646,28.08,80,100,7.44,MH,1699739711
4,4,llanes,43.4198,-4.7549,17.12,88,100,2.38,ES,1699739711


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [11]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    color = "City",
    alpha = 0.7,
    size = "Humidity",
    geo = True,
    tiles = "OSM"
)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [23]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_clean = city_data_df[(city_data_df['Max Temp'] <= 27) & (city_data_df['Max Temp'] > 21)]

# Drop any rows with null values
city_data_clean = city_data_clean.dropna(how='any')

# Display sample data
city_data_clean

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,1,coronel suarez,-37.4547,-61.9334,21.42,45,1,4.06,AR,1699739709
7,7,nova sintra,14.8667,-24.7167,22.87,70,50,4.45,CV,1699739713
15,15,adamstown,-25.0660,-130.1015,22.96,81,27,5.96,PN,1699739717
17,17,acarau,-2.8856,-40.1200,26.10,76,0,7.56,BR,1699739720
21,21,hengchun,22.0042,120.7439,24.72,80,21,6.87,TW,1699739722
...,...,...,...,...,...,...,...,...,...,...
558,558,kasongo-lunda,-6.4667,16.8167,21.14,98,69,0.48,CD,1699740036
559,559,calingasta,-31.3308,-69.4078,24.87,8,0,11.16,AR,1699740037
560,560,beau vallon,-20.4189,57.6953,23.45,88,75,1.54,MU,1699740038
567,567,am timan,11.0297,20.2827,26.15,63,22,2.50,TD,1699740044


### Step 3: Create a new DataFrame called `hotel_df`.

In [30]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_clean[['City', 'Country', 'Lat', 'Lng', 'Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ' '

# Display sample data
hotel_df

C:\Users\Chad\AppData\Local\Temp\ipykernel_12792\4260431133.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = ' '


,City,Country,Lat,Lng,Humidity,Hotel Name
1,coronel suarez,AR,-37.4547,-61.9334,45,
7,nova sintra,CV,14.8667,-24.7167,70,
15,adamstown,PN,-25.0660,-130.1015,81,
17,acarau,BR,-2.8856,-40.1200,76,
21,hengchun,TW,22.0042,120.7439,80,
...,...,...,...,...,...,...
558,kasongo-lunda,CD,-6.4667,16.8167,98,
559,calingasta,AR,-31.3308,-69.4078,8,
560,beau vallon,MU,-20.4189,57.6953,88,
567,am timan,TD,11.0297,20.2827,63,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [36]:
# Set parameters to search for a hotel
radius = 5000
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key,
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lng},{lat},{radius}'
    params["bias"] = f'proximity:{lng},{lat}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params).json()
    
    # Convert the API response to JSON format
    # name_address = # YOUR CODE HERE
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
coronel suarez - nearest hotel: La Casa Hotel Boutique
nova sintra - nearest hotel: Residência Ka Dencho
adamstown - nearest hotel: No hotel found
acarau - nearest hotel: No hotel found
hengchun - nearest hotel: 海的顏色精品旅館
avarua - nearest hotel: Paradise Inn
bonthe - nearest hotel: No hotel found
eraan - nearest hotel: No hotel found
sorong - nearest hotel: Hotel Membramo Marina
georgetown - nearest hotel: Page 63 hostel
namibe - nearest hotel: Hotel Chik Chik Namibe
touros - nearest hotel: Pousada Atlântico
takoradi - nearest hotel: Hillcrest
waddan - nearest hotel: No hotel found
mont-dore - nearest hotel: No hotel found
eyl - nearest hotel: فندق ايل دابي
inhambane - nearest hotel: Hotel de Inhambane
antonio enes - nearest hotel: Hotel Quirimbas
mana - nearest hotel: Le Samana
adjumani - nearest hotel: Bucolic Hotel
da nang - nearest hotel: Thanh Thanh Hotel
luanda - nearest hotel: Hotel Forum
port mathurin - nearest hotel: Escale Vacances
west island - nearest h

,City,Country,Lat,Lng,Humidity,Hotel Name
1,coronel suarez,AR,-37.4547,-61.9334,45,La Casa Hotel Boutique
7,nova sintra,CV,14.8667,-24.7167,70,Residência Ka Dencho
15,adamstown,PN,-25.0660,-130.1015,81,No hotel found
17,acarau,BR,-2.8856,-40.1200,76,No hotel found
21,hengchun,TW,22.0042,120.7439,80,海的顏色精品旅館
...,...,...,...,...,...,...
558,kasongo-lunda,CD,-6.4667,16.8167,98,No hotel found
559,calingasta,AR,-31.3308,-69.4078,8,Nora Hôtel
560,beau vallon,MU,-20.4189,57.6953,88,Le Jardin de Beau Vallon
567,am timan,TD,11.0297,20.2827,63,Nimla


In [37]:
hotel_df['Hotel Name'].value_counts()

Hotel Name
No hotel found             51
Ballina Beach Resort        2
Nana's Lodge                1
Embassy Suites              1
Whitsunday Sands Resort     1
                           ..
Bella Casa Hotel            1
Bon Hotel Waterfront        1
Kharga Hotel                1
hotel obino                 1
Nimla                       1
Name: count, Length: 113, dtype: int64

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [58]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    hover_cols=['Hotel Name', 'Country'],
    color = "City",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    alpha = 0.7,
    geo = True,
    tiles = "OSM"
)

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)